In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
from plotnine import *
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
# load in pre-cleaned deaths data for Texas and comparison states
deaths = pd.read_csv("../20_intermediate_files/deaths_wa.csv")


In [3]:
# two separate dfs - one for texas and one for comp states

washington = deaths[deaths["StateName"] == "Washington"]
comp = deaths[deaths["StateName"] != "Washington"]

### Helper functions 

In [4]:
# function to select only overdose records
# double check this

def select_overdose(record):
    """Simple function to select only overdose records"""

    if record == "All other non-drug and non-alcohol causes":
        return 0

    if record == "All other alcohol-induced causes":
        return 0

    if record == "All other drug-induced causes":
        return 0

    if record == "Alcohol poisonings (overdose) (X45, X65, Y15)":
        return 0

    if record == "Drug poisonings (overdose) Unintentional (X40-X44)":
        return 1

    if record == "Drug poisonings (overdose) Suicide (X60-X64)":
        return 1

    if record == "Drug poisonings (overdose) Undetermined (Y10-Y14)":
        return 1

    else:
        return "error"
    

# copy to fix the dreaded "A value is trying to be set on a copy of a slice" error
wa_deaths = washington.copy()
comp_deaths = comp.copy()

# apply new function to our df
wa_deaths["overdose"] = wa_deaths["Drug/Alcohol Induced Cause"].apply(lambda x: select_overdose(x))
comp_deaths["overdose"] = comp_deaths["Drug/Alcohol Induced Cause"].apply(lambda x: select_overdose(x))

assert len(wa_deaths[wa_deaths["overdose"] == "error"]) == 0
assert len(comp_deaths[comp_deaths["overdose"] == "error"]) == 0

# filter accordingly based on new column
wa_deaths = wa_deaths[wa_deaths["overdose"] != 0]
comp_deaths = comp_deaths[comp_deaths["overdose"] != 0]

In [5]:
# calculate overdoses per 100_000 residents

wa_deaths["overdose_per_100k"] = wa_deaths["Deaths"] / wa_deaths["Population"] * 100_000
comp_deaths["overdose_per_100k"] = comp_deaths["Deaths"] / comp_deaths["Population"] * 100_000

In [6]:
# groupby year and county
wa_result = wa_deaths.groupby(["Year", "County"])["overdose_per_100k"].sum().reset_index()
comp_result = comp_deaths.groupby(["Year", "StateName", "CountyName"])["overdose_per_100k"].sum().reset_index()

In [7]:
wa_result

,Year,County,overdose_per_100k
0,2009.0,Benton,9.934433
1,2009.0,Clallam,18.290023
2,2009.0,Clark,14.006400
3,2009.0,Cowlitz,17.625286
4,2009.0,Grant,19.464163
...,...,...,...
105,2015.0,Snohomish,16.499978
106,2015.0,Spokane,18.828923
107,2015.0,Thurston,9.702289
108,2015.0,Whatcom,10.857456


In [21]:
# calculate summary stats
wa_res = pd.DataFrame(wa_result.describe()["overdose_per_100k"]).rename(columns={"overdose_per_100k": "Opioid Overdose Deaths Per 100K Residents - Washington"})
comp_res = pd.DataFrame(comp_result.describe()["overdose_per_100k"]).rename(columns={"overdose_per_100k": "Opioid Overdose Deaths Per 100K Residents - Control States (MD,NC,CO)"})

In [22]:
stats = pd.concat([wa_res, comp_res], axis=1)
stats

,Opioid Overdose Deaths Per 100K Residents - Washington,"Opioid Overdose Deaths Per 100K Residents - Control States (MD,NC,CO)"
count,7.000000,7.000000
mean,13.668151,14.953933
std,0.822056,2.003544
min,12.020064,12.511895
25%,13.433789,13.979726
50%,13.984919,14.162456
75%,14.226162,15.639365
max,14.352174,18.764995


In [10]:
# overwrite wa result to be grouped by year only
wa_result = wa_result.groupby("Year")["overdose_per_100k"].mean().reset_index()
comp_result = comp_result.groupby(["Year"])["overdose_per_100k"].mean().reset_index()

In [11]:
wa_res_checkpoint = wa_result.copy()
comp_res_checkpoint = comp_result.copy()


# assert length is the same
assert (len(wa_result) == len(wa_res_checkpoint)) & (len(comp_result) == len(comp_res_checkpoint))

## Pre-post plot - overdoses per 100k through the years

In [12]:

wa_b4 = wa_result[wa_result["Year"] < 2012]
wa_after = wa_result[wa_result["Year"] >= 2012]

In [13]:
comp_b4 = comp_result[comp_result["Year"] < 2012]
comp_after = comp_result[comp_result["Year"] >= 2012]


In [14]:
x = "Year"
y = "overdose_per_100k"

In [27]:
# Function to plot a vertical line at year of policy implementation
def vertical_line(year):
   
    line = alt.Chart(pd.DataFrame({
    'Year': [year],
    'color': ["black"]
    })).mark_rule().encode(
    x='Year:Q',
    color=alt.Color('color:N', scale=None)
    )
 
    return line


In [28]:
# creating the vertical line at 2012, the year of implementation of policy changes in Washington
line = vertical_line(2012)


In [29]:
# creating the regression model and calculating the error bands for creating the plots
 
def get_reg_fit_and_ci(data, color, xvar, yvar, legend, alpha=0.05):
   
    # Creating the grid for predicted values
    colour = color
    years = [2009,2010,2011,2012,2013,2014,2015]
    x = data.loc[pd.notnull(data[yvar]), xvar]
    xmin = x.min()
    xmax = x.max()
    step = (xmax - xmin) / 100
    grid = np.arange(xmin, xmax + step, step)
    predictions = pd.DataFrame({xvar: grid})
 
    # Fitting the model and making the predictions
    model = smf.ols(f"{yvar} ~ {xvar}", data=data).fit()
    model_predict = model.get_prediction(predictions[xvar])
    predictions[yvar] = model_predict.summary_frame()["mean"]
    predictions[["ci_low", "ci_high"]] = model_predict.conf_int(alpha=alpha)
   
    # Utilizing the predictions to create the points and error bands in the chart
    predictions["Before/After"] = f"{legend}"
    reg = (
        alt.Chart(predictions)
        .mark_line()
        .encode(
            x=xvar,
            y=alt.Y(yvar),
            color = alt.Color("Before/After",legend=alt.Legend(title = "States"))
        )
    )
 
    ci = (
        alt.Chart(predictions)
        .mark_errorband()
        .encode(
            alt.X(f"{xvar}:Q", axis=alt.Axis(format=".0f", values=years)),
            y=alt.Y(
                "ci_low",
                title="Opioid Overdose Deaths Per 100K Residents",
                scale=alt.Scale(zero=False),
            ),
            y2="ci_high",
            color=alt.value(f"{color}"),
        )
    )
    chart = ci + reg
    return predictions, chart
 
 


In [30]:
# Using the get_reg_fit_and_ci function to create the charts
 
def build_chart(data, color, xvar, yvar, legend, alpha=0.05):
    fit, reg_chart = get_reg_fit_and_ci(
        data=data, color=color, xvar=xvar, yvar=yvar,  legend=legend, alpha=alpha,
    )
    return reg_chart

In [32]:
# creating the final plot for pre-post analysis of drug overdose deaths in Florida
 
washington_before_chart = build_chart(
    wa_b4, "orange", "Year","overdose_per_100k", "Washington",  alpha=0.05
)
 
washington_after_chart = build_chart(
    wa_after, "orange", "Year", "overdose_per_100k", "Washington", alpha=0.05
)

other_states_before_chart = build_chart(
    comp_b4, "blue", "Year","overdose_per_100k", "Control States - CO,MD,NC",  alpha=0.05
)
 
other_states_after_chart = build_chart(
    comp_after, "blue", "Year", "overdose_per_100k", "Control States - CO,MD,NC", alpha=0.05
)
 
washington_final_diff_in_diff_deaths = (washington_before_chart + washington_after_chart + other_states_before_chart + other_states_after_chart + line).properties(title="Difference-In-Difference Analysis Of Opioid Overdose Deaths In Washington")
 
washington_final_diff_in_diff_deaths

c:\Users\annap\miniconda3\lib\site-packages\altair\utils\core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.


alt.LayerChart(...)